# 데이터 불러오기

- 간단하게 데이터를 보면서 이상이 있는 데이터를 점검해 준다

In [2]:
import pandas as pd
import numpy as np
import warnings

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',10)
warnings.simplefilter('ignore')

file_path = 'D:\\mydata\\데이터분석\\FastCampus_전처리\\part-5.-머신러닝-모델의-성능-향상을-위한-전처리\\5. 머신러닝 모델의 성능 향상을 위한 전처리\\데이터\\'

In [4]:
df = pd.read_csv(file_path+'Telco_churn_prediction.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [10]:
for col in df.columns:
    print(f'{col} : {sorted(df[col].unique())[:10]} ::: {len(df[col].unique())}')

customerID : ['0002-ORFBO', '0003-MKNFE', '0004-TLHLJ', '0011-IGKFF', '0013-EXCHZ', '0013-MHZWF', '0013-SMEOE', '0014-BMAQU', '0015-UOCOJ', '0016-QLJIS'] ::: 7043
gender : ['Female', 'Male'] ::: 2
SeniorCitizen : [0, 1] ::: 2
Partner : ['No', 'Yes'] ::: 2
Dependents : ['No', 'Yes'] ::: 2
tenure : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] ::: 73
PhoneService : ['No', 'Yes'] ::: 2
MultipleLines : ['No', 'No phone service', 'Yes'] ::: 3
InternetService : ['DSL', 'Fiber optic', 'No'] ::: 3
OnlineSecurity : ['No', 'No internet service', 'Yes'] ::: 3
OnlineBackup : ['No', 'No internet service', 'Yes'] ::: 3
DeviceProtection : ['No', 'No internet service', 'Yes'] ::: 3
TechSupport : ['No', 'No internet service', 'Yes'] ::: 3
StreamingTV : ['No', 'No internet service', 'Yes'] ::: 3
StreamingMovies : ['No', 'No internet service', 'Yes'] ::: 3
Contract : ['Month-to-month', 'One year', 'Two year'] ::: 3
PaperlessBilling : ['No', 'Yes'] ::: 2
PaymentMethod : ['Bank transfer (automatic)', 'Credit card (automa

# Train, Test 나누기

In [11]:
X = df.drop(['customerID','Churn'], axis=1)
Y = df['Churn']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y)

In [12]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5282 entries, 6055 to 4265
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5282 non-null   object 
 1   SeniorCitizen     5282 non-null   int64  
 2   Partner           5282 non-null   object 
 3   Dependents        5282 non-null   object 
 4   tenure            5282 non-null   int64  
 5   PhoneService      5282 non-null   object 
 6   MultipleLines     5282 non-null   object 
 7   InternetService   5282 non-null   object 
 8   OnlineSecurity    5282 non-null   object 
 9   OnlineBackup      5282 non-null   object 
 10  DeviceProtection  5282 non-null   object 
 11  TechSupport       5282 non-null   object 
 12  StreamingTV       5282 non-null   object 
 13  StreamingMovies   5282 non-null   object 
 14  Contract          5282 non-null   object 
 15  PaperlessBilling  5282 non-null   object 
 16  PaymentMethod     5282 non-null   objec

In [16]:
# 위에서 봤던 내용중 TotalCharges가 이상한 점을 고쳐준다
x_train['TotalCharges'].replace(' ', np.nan, inplace=True)
x_test['TotalCharges'].replace(' ', np.nan, inplace=True)

x_train['TotalCharges'] = x_train['TotalCharges'].astype(float)
x_test['TotalCharges'] = x_test['TotalCharges'].astype(float)

In [17]:
x_train['TotalCharges'].dtype

dtype('float64')

# 범주형 변수 찾고 바꾸기

In [18]:
category_cols = [col for col in x_train.columns if x_train[col].dtype==object]
category_cols

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [19]:
def make_suffix(df):
    import pandas as pd
    
    col_names = []
    suffix_names = []
    
    for col in df.columns:
        values = sorted(df[col].unique())[1:]
        suffix_names += values
        col_names += [col]*(len(values))
        
    return list(pd.Series(col_names)+'_'+pd.Series(suffix_names))

In [25]:
from sklearn.preprocessing import OneHotEncoder as OHE

dummy_model = OHE(drop='first', sparse=False).fit(x_train[category_cols])

x_train_ohe = pd.DataFrame(dummy_model.transform(x_train[category_cols]), columns = make_suffix(x_train[category_cols]))
x_test_ohe = pd.DataFrame(dummy_model.transform(x_test[category_cols]), columns = make_suffix(x_test[category_cols]))

x_train_ohe.index = x_train.index
x_test_ohe.index = x_test.index

x_train = pd.concat([x_train.drop(category_cols, axis=1), x_train_ohe], axis=1)
x_test = pd.concat([x_test.drop(category_cols, axis=1), x_test_ohe], axis=1)

In [26]:
x_train.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,...,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
6055,0,6,55.00,340.4,0.0,...,0.0,1.0,0.0,1.0,0.0
1835,0,1,45.00,45.0,0.0,...,0.0,1.0,0.0,1.0,0.0
5305,0,43,78.80,3460.3,0.0,...,1.0,1.0,0.0,1.0,0.0
694,1,22,20.05,417.0,0.0,...,0.0,1.0,0.0,0.0,1.0
1749,1,52,110.75,5832.0,1.0,...,0.0,1.0,0.0,1.0,0.0


# 결측값 처리

In [29]:
from sklearn.impute import KNNImputer

KI = KNNImputer(n_neighbors = 5)
KI.fit(x_train)

x_train = pd.DataFrame(KI.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(KI.transform(x_test), columns=x_test.columns)

In [30]:
x_train.isna().sum()

SeniorCitizen                            0
tenure                                   0
MonthlyCharges                           0
TotalCharges                             0
gender_Male                              0
Partner_Yes                              0
Dependents_Yes                           0
PhoneService_Yes                         0
MultipleLines_No phone service           0
MultipleLines_Yes                        0
InternetService_Fiber optic              0
InternetService_No                       0
OnlineSecurity_No internet service       0
OnlineSecurity_Yes                       0
OnlineBackup_No internet service         0
OnlineBackup_Yes                         0
DeviceProtection_No internet service     0
DeviceProtection_Yes                     0
TechSupport_No internet service          0
TechSupport_Yes                          0
StreamingTV_No internet service          0
StreamingTV_Yes                          0
StreamingMovies_No internet service      0
StreamingMo

# 연속형, 이산형 변수 분리

In [31]:
cont_cols = [col for col in x_train.columns if len(x_train[col].unique()) > 3]
cate_cols = [col for col in x_train.columns if len(x_train[col].unique()) <= 3] 

In [32]:
cont_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

# 이상값 처리

In [37]:
def IQR_Rule(arr):
    q1 = np.quantile(arr, 0.25)
    q3 = np.quantile(arr, 0.75)
    IQR = q3-q1
    
    not_outliers_cond = (q1-1.5*IQR < arr) & (q3+1.5*IQR > arr)
    return not_outliers_cond

num_outliers = len(x_train) - x_train[cont_cols].apply(IQR_Rule).sum(axis=0)
print(num_outliers)

tenure            0
MonthlyCharges    0
TotalCharges      0
dtype: int64


# 라벨 분포 확인 및 under / over sampling 결정

In [44]:
y_train.value_counts()

No     3887
Yes    1395
Name: Churn, dtype: int64

In [46]:
y_train.replace({'No':-1, 'Yes':1}, inplace=True)
y_test.replace({'No':-1, 'Yes':1}, inplace=True)

In [47]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report

kNN_model = KNN(n_neighbors=5).fit(x_train, y_train)
y_pred = kNN_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.81      0.89      0.84      1287
           1       0.58      0.42      0.49       474

    accuracy                           0.76      1761
   macro avg       0.69      0.65      0.67      1761
weighted avg       0.74      0.76      0.75      1761



# 자기상관성 처리

In [48]:
from sklearn.linear_model import LinearRegression as LR

VIF_dict = dict()

for col in x_train.columns:
    model = LR().fit(x_train.drop(col, axis=1), x_train[col])
    r2 = model.score(x_train.drop(col, axis=1), x_train[col])
    VIF = 1/(1-r2)
    VIF_dict[col] = VIF

VIF_dict

{'SeniorCitizen': 1.1499468523689165,
 'tenure': 7.459827433693643,
 'MonthlyCharges': 864.8671144621196,
 'TotalCharges': 10.777548214085673,
 'gender_Male': 1.0031933180149726,
 'Partner_Yes': 1.4715067736960434,
 'Dependents_Yes': 1.3904162883739417,
 'PhoneService_Yes': inf,
 'MultipleLines_No phone service': inf,
 'MultipleLines_Yes': 7.316938107307301,
 'InternetService_Fiber optic': 149.062072451942,
 'InternetService_No': inf,
 'OnlineSecurity_No internet service': inf,
 'OnlineSecurity_Yes': 6.366915527469948,
 'OnlineBackup_No internet service': inf,
 'OnlineBackup_Yes': 6.771951594159919,
 'DeviceProtection_No internet service': inf,
 'DeviceProtection_Yes': 6.95170176049174,
 'TechSupport_No internet service': inf,
 'TechSupport_Yes': 6.508675028003128,
 'StreamingTV_No internet service': inf,
 'StreamingTV_Yes': 24.250919287001807,
 'StreamingMovies_No internet service': inf,
 'StreamingMovies_Yes': 24.158351314751492,
 'Contract_One year': 1.617380770975177,
 'Contract_Tw

In [49]:
remove_cols = [key for key in VIF_dict.keys() if VIF_dict[key]>30]
remove_cols

['MonthlyCharges',
 'PhoneService_Yes',
 'MultipleLines_No phone service',
 'InternetService_Fiber optic',
 'InternetService_No',
 'OnlineSecurity_No internet service',
 'OnlineBackup_No internet service',
 'DeviceProtection_No internet service',
 'TechSupport_No internet service',
 'StreamingTV_No internet service',
 'StreamingMovies_No internet service']

In [50]:
r_x_train = x_train.drop(remove_cols, axis=1)
r_x_test = x_test.drop(remove_cols, axis=1)

cont_cols = list(set(cont_cols)-set(remove_cols))
cate_cols = list(set(cate_cols)-set(remove_cols))

# 왜도 처리

In [55]:
x_train[cont_cols].skew()

tenure          0.232309
TotalCharges    0.957825
dtype: float64

In [63]:
# 왜도가 1.5 이상 컬럼 처리
biased_cols = x_train[cont_cols].columns[x_train[cont_cols].skew().abs() > 1.5]
x_train_bias = x_train[biased_cols] - x_train[biased_cols].min()+1
x_train_bias = np.log10(x_train_bias)

# 스케일링

In [65]:
from sklearn.preprocessing import MinMaxScaler

MMS = MinMaxScaler().fit(x_train)
s_x_train = pd.DataFrame(MMS.transform(x_train), columns=x_train.columns)
s_x_test = pd.DataFrame(MMS.transform(x_test), columns=x_test.columns)

# 변수 선택 및 파라미터 그리드 튜닝

In [70]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LGR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.feature_selection import chi2, f_classif
from sklearn.model_selection import ParameterGrid as PG

model_params_grid = dict()

model_params_grid[RFC] = PG({'max_depth':[2,3,4,5,6],
                             'n_estimators':[5,10,15,25,50,100]})
model_params_grid[DTC] = PG({'max_depth':[2,3,4,5,6],
                             'min_samples_leaf':[1,2,3,4,5]})
model_params_grid[LGR] = PG({'C':[0.0001, 0.001, 0.005, 0.01, 0.05, 0.1],
                             'max_iter':[10,100,1000,10000,100000]})
model_params_grid[KNN] = PG({'n_neighbors':[3,5,7,9,11]})

In [72]:
cont_cols_pval = f_classif(s_x_train[cont_cols], y_train)[1]
cate_cols_pval = chi2(s_x_train[cate_cols], y_train)[1]

cont_pvals = pd.Series(cont_cols_pval, index=cont_cols)
cate_pvals = pd.Series(cate_cols_pval, index=cate_cols)

pvals = pd.concat([cont_pvals, cate_pvals])
pvals.sort_values(ascending=True, inplace=True)

pvals

tenure                                   3.654332e-154
Contract_Two year                         2.418767e-82
PaymentMethod_Electronic check            4.231311e-71
TotalCharges                              8.405870e-47
Contract_One year                         8.491780e-31
TechSupport_Yes                           2.746091e-26
OnlineSecurity_Yes                        1.150619e-25
SeniorCitizen                             4.926435e-23
Dependents_Yes                            2.845288e-22
PaymentMethod_Credit card (automatic)     8.185773e-21
PaperlessBilling_Yes                      2.064023e-17
Partner_Yes                               4.712542e-14
PaymentMethod_Mailed check                2.383749e-08
OnlineBackup_Yes                          9.881768e-07
DeviceProtection_Yes                      8.002919e-05
StreamingTV_Yes                           4.105794e-04
StreamingMovies_Yes                       7.514375e-04
MultipleLines_Yes                         5.608627e-02
gender_Mal

In [75]:
best_score = 0
for k in range(len(s_x_train.columns),5,-1):
    print(k)
    ss_x_train = s_x_train[pvals.iloc[:k].index]
    ss_x_test = s_x_test[pvals.iloc[:k].index]
    for M in model_params_grid.keys():
        for P in model_params_grid[M]:
            model = M(**P).fit(ss_x_train, y_train)
            y_pred = model.predict(ss_x_test)
            score = f1_score(y_test, y_pred, average='weighted')
            
            if score > best_score:
                best_score = score.mean()
                best_model = M
                best_params = P
                best_feature = ss_x_train.columns
                report = classification_report(y_test, y_pred)

30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6


# 결과 확인

In [76]:
print(f'best_score : {best_score}')
print(f'best_model : {best_model}')
print(f'best_params : {best_params}')
print(f'best_features : {best_feature}')
print('='*50)
print(report)

best_score : 0.7851572746580283
best_model : <class 'sklearn.linear_model._logistic.LogisticRegression'>
best_params : {'C': 0.1, 'max_iter': 100}
best_features : Index(['tenure', 'Contract_Two year', 'PaymentMethod_Electronic check',
       'TotalCharges', 'Contract_One year', 'TechSupport_Yes',
       'OnlineSecurity_Yes', 'SeniorCitizen', 'Dependents_Yes',
       'PaymentMethod_Credit card (automatic)', 'PaperlessBilling_Yes',
       'Partner_Yes', 'PaymentMethod_Mailed check', 'OnlineBackup_Yes',
       'DeviceProtection_Yes', 'StreamingTV_Yes', 'StreamingMovies_Yes',
       'MultipleLines_Yes', 'gender_Male'],
      dtype='object')
              precision    recall  f1-score   support

          -1       0.82      0.92      0.87      1287
           1       0.69      0.46      0.55       474

    accuracy                           0.80      1761
   macro avg       0.76      0.69      0.71      1761
weighted avg       0.79      0.80      0.79      1761



# 최종 분류/예측치 내보내기

In [80]:
lgr_model = LGR(C=0.1, max_iter=100).fit(s_x_train[best_feature], y_train)
y_pred = lgr_model.predict(s_x_test[best_feature])
print(accuracy_score(y_pred, y_test))
print(f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_pred, y_test))

0.7989778534923339
0.8127984323266394
              precision    recall  f1-score   support

          -1       0.92      0.82      0.87      1441
           1       0.46      0.69      0.55       320

    accuracy                           0.80      1761
   macro avg       0.69      0.76      0.71      1761
weighted avg       0.84      0.80      0.81      1761



In [83]:
last_x_train = pd.concat([s_x_train[best_feature], s_x_test[best_feature]], axis=0)
last_y_train = pd.concat([y_train, y_test], axis=0)

In [ ]:
'''
lgr_model = LGR(C=0.1, max_iter=100).fit(last_x_train, last_y_train)
last_y_pred = lgr_model.predict(test_data)
last_y_pred.to_csv('last_result.csv', index=False)
'''